In [3]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-21 04:07:58--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.54MB/s    in 0.2s    

2022-12-21 04:07:58 (5.54 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [7]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [8]:
# 1 create a new DataFrame with the rows where total_votes count>=20 
helpfulvotes_df = vine_df[vine_df["total_votes"]>=20]
helpfulvotes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [9]:
# Load in a sql funtion to use columns
from pyspark.sql.functions import col

In [10]:
#Filter the new DataFrame to create a new DataFrame with all the rows where (helpful_votes/total_votes)>= 50%
helpful_percent_df = helpfulvotes_df.withColumn('percent_helful', col("helpful_votes")/col("total_votes"))
helpful_percent_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|      percent_helful|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|  0.6176470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|                 0.6|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|                0.84|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y| 0.45161290322580644|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|  0.9166666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|  0.7846153846153846|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|  0.8611111111111112|
| R6KTC1OP

In [11]:
#Filter the new df to retrieve rows where vine == 'Y'.
paid_reviews_df = helpful_percent_df.filter(helpful_percent_df["vine"] == "Y")
paid_reviews_df.show() 

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_helful|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N| 0.8888888888888888|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|                1.0|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N| 0.7916666666666666|
|R371M3ZU3ULQHM|          1|            0|         25|   Y|                N|                0.0|
|R18YQ4NRL2373C|          3|            2|         28|   Y|                N|0.07142857142857142|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N| 0.8461538461538461|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N| 0.7692307692307693|
|R286MFBAJ8NPD6|    

In [12]:
unpaid_reviews_df = helpful_percent_df.filter(helpful_percent_df["vine"] == "N")
unpaid_reviews_df.show() 

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|      percent_helful|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|  0.6176470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|                 0.6|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|                0.84|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y| 0.45161290322580644|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|  0.9166666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|  0.7846153846153846|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|  0.8611111111111112|
| R6KTC1OP

In [13]:
from pyspark.sql.functions import count

In [15]:
total_votes_unfiltered = vine_df.count()
total_votes_unfiltered

1785997

In [16]:
helpfulvotes_df_count = helpfulvotes_df.count()
helpfulvotes_df_count

65379

In [60]:
# Number of 5-star Vine reviews
paid_5star = paid_reviews_df[paid_reviews_df["star_rating"] == "5"].count()
paid_5star

48

In [61]:
# Total number of Vine reviews
total_paid_reviews = paid_reviews_df.count()
total_paid_reviews

104

In [64]:
# Percentage of 5-star Vine reviews
paid_5star_percentage = paid_5star / total_paid_reviews * 100
paid_5star_percentage

46.15384615384615

In [63]:
# Number of 5-star non-Vine reviews
unpaid_5star = unpaid_reviews_df[unpaid_reviews_df["star_rating"] == "5"].count()
unpaid_5star

20439

In [66]:
# Total number of non-Vine reviews
total_unpaid_reviews = unpaid_reviews_df.count()
total_unpaid_reviews

65275

In [67]:
# Percentage of 5-star non-Vine reviews
unpaid_5satr_percentage = unpaid_5star / total_unpaid_reviews * 100
unpaid_5satr_percentage

31.312140942167755